In [ ]:
from fastai.vision import *
from pathlib import Path

In [ ]:
path_img = Path("../data/aligned_rus/shokugeki_no_soma/31")
target_img = Path("../data/aligned_eng/shokugeki_no_soma/31")

In [ ]:
mnist = untar_data(URLs.MNIST_TINY)
tfms = get_transforms(do_flip=False)

In [ ]:
get_y_fn = lambda x: target_img/f'{x.stem}.jpg'

In [ ]:
data = (ImageImageList.from_folder(path_img)
        #Where to find the data? -> in path_img and its subfolders
        .split_by_rand_pct()
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_func(get_y_fn)
        #How to label? -> use the label function on the file name of the data
        .transform(get_transforms(), tfm_y=True, size=1024)
        #Data augmentation? -> use tfms with a size of 128, also transform the label images
        .databunch(bs=8))

In [ ]:
data.show_batch(rows=1, figsize=(10,10))

In [ ]:
data.train_ds.y[1]

In [ ]:
idx = 5
x = data.train_ds.x[idx].data.clone()
y = data.train_ds.y[idx].data.clone()
x_mean = x.mean(dim=(1,2), keepdim=True)
y_mean = y.mean(dim=(1,2), keepdim=True)
x_std = x.std(dim=(1,2), keepdim=True)
y_std = y.std(dim=(1,2), keepdim=True)

x -= x_mean
y -= y_mean
x /= x_std
y /= y_std
_,axs = plt.subplots(1,3, figsize=(20,20))
Image((x*y_std + y_mean).clamp(0, 1)).show(ax=axs[0])
data.train_ds.y[idx].show(ax=axs[1])
diff = (x - y).abs()
diff *= (diff > 2.2)
diff /= diff.max()
Image(diff).show(ax=axs[2])
x.shape